In [2]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

#  SVM module
from PredictSVM import PredictMultiState
from PredictSVM import get_sessions, load_trial_times_whole_stack

from PredictSVM import PredictSVMTime

# 
data_dir = '/media/cat/4TBSSD/yuki/'
#data_dir = '/mnt/f418adc0-4ff9-410c-93c9-d6d21a1e9c35/yuki/'
#data_dir = '/media/cat/1TB/yuki/testdata/'

labels = ['left_paw', 
            'right_paw',
            'nose',
            'jaw',
            'right_ear',
            'tongue',
            'lever',
            'all',
            'code_04',
            'code_04_lockout']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [4]:
######################################################
##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
######################################################

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
names = ['IJ2']

# 
svm = PredictMultiState()
svm.root_dir = data_dir
svm.parallel = True
svm.n_cores = 30

# 
svm.no_movement = 3            # number of sec intra-feature lockout
svm.window = 15                # number of seconds pre movement to compute classification
svm.sliding_window = 30         # number of time steps that go into classifier
svm.sliding_window_step = 1    # number of frames to step the sliding window across
svm.code = 'multi_class'
svm.xvalidation = 10
svm.all_comps = 30

#
svm.animal_id = names[0]
svm.session_id = 'Feb1_'


#
svm.classifier = 'svm'               # 'svm',  'random_forest'
svm.features_selected = [0,5]        # ['left_paw',          0
                                     #  'right_paw',         1
                                     #  'nose',              2
                                     #  'jaw',               3 
                                     #  'right_ear',         4 
                                     #  'tongue',            5
                                     #  'lever',             6
                                     
                                     #'all',                 7   NOT CODED
                                     #'code_04',             8
                                     #'code_04_lockout']     9

#                                        
svm.inter_feature_lockout_window = 0
                                        

# can add this to aut
svm.get_sessions()
#
print (svm.sessions)

#
svm.load_trials()

# convert data to train and test datasets;
svm.prepare_and_run_classifier()

# print (svm.acc)

# 
fig=plt.figure()
acc = np.array(svm.acc)
mean = acc.mean(1)
std = np.std(acc,1)
t = np.arange(mean.shape[0])/30.-14
plt.fill_between(t,mean+std, mean-std, alpha=.2)
plt.plot(t,mean)

# 
plt.ylim(bottom=0)
#plt.xlim(-5,5)
plt.plot([t[0],t[-1]],[1./len(svm.features_selected),1./len(svm.features_selected)],'--',c='black')
plt.title(labels[svm.features_selected[0]]+ " "+labels[svm.features_selected[1]])
plt.show()


['IJ2pm_Feb1_30Hz']
  running:  IJ2pm_Feb1_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  -9.600000000000001
Trials stack (make sure same across D1):  (2, 38, 900, 30)
0  size:  (38, 900, 30)
1  size:  (38, 900, 30)


904it [01:53,  7.94it/s]                         


In [33]:
##########################################################
########## MOST SIGNIFICANT COEFFICIENT ANALYSIS #########
##########################################################

# print (len(svm.coef))
# print (np.array(svm.coef[1]).shape)
# ctr=1
# arr = []
# for k in range(0,len(svm.coef)-30,1):
# #     ax=plt.subplot(5,6,ctr)
#     temp = np.array(svm.coef[k]).squeeze().mean(0).reshape(30,30).T
#     arr.append(temp)

# # most significant analyais
# arr = np.array(arr)
# t = np.arange(arr.shape[0])/30-14
# for k in range(arr.shape[1]):
#     for p in range(arr.shape[2]):
#         plt.plot(t,arr[:,k,p],alpha=.7)
# plt.show()

In [5]:
# ###############################################
# ############ LEVER TO BODY CORRELATION ########
# ###############################################
# import pycorrelate
# t = svm.code_04_times_unshifted[:,1]

# fig=plt.figure()
# bins = np.arange(-10,10,1/15)
# for k in range(7):
#     ax=plt.subplot(3,3,k+1)
#     u =  np.array(svm.features_unshifted[k])[:,1]
#     res = pycorrelate.pcorrelate(t, u, bins=bins)

#     plt.plot(bins[:-1],res)
#     argmax = np.argmax(res)
#     print (" shift: ", bins[argmax])
#     plt.title(labels[k]+ " "+str(round(bins[argmax],2))+"sec")
# plt.suptitle("Correlation between body and lever ")
# plt.show()


 shift:  -9.733333333333334
 shift:  -9.600000000000001
 shift:  0.5999999999999623
 shift:  -9.866666666666667
 shift:  3.5999999999999517
 shift:  -9.933333333333334
 shift:  3.799999999999951


In [6]:
##############################################
############ BODY TO BODY CORRELATION ########
##############################################
fig=plt.figure()
bins = np.arange(-10,10,.1)
ctr=1
bins = np.arange(-10,10,1/15.)
for k in range(7):
    for p in range(8):
        ax=plt.subplot(7,8,ctr)
        t = np.array(svm.features_unshifted[k])[:,1]
        if p==7:
            u =  code_04[:,1]  
        else:
            u =  np.array(svm.features_unshifted[p])[:,1]
        res = pycorrelate.pcorrelate(t, u, bins=bins)

        plt.plot(bins[:-1],res)
        argmax = np.argmax(res)

        if k==0:
            plt.title(labels[p]+ " " + str(round(bins[argmax],2))+"sec",fontsize=10,pad=.9)
        else:
            plt.title(str(round(bins[argmax],2))+"sec",fontsize=10,pad=.9)
            
            
        if p==0:
            plt.ylabel(labels[k],fontsize=10)
        if k!=6:
            plt.xticks([])
        plt.yticks([])
        
        std = np.std(res)
        plt.plot([bins[0],bins[-1]],[std,std],'--',c='black')
        
        ctr+=1
        
plt.suptitle("Correlation between body and lever ")
plt.show()


NameError: name 'code_04' is not defined

In [8]:

    
    
# 


IA1pm_Feb1_30Hz
Lever to [Ca] shift:  2.566666666666667
DLC to [Ca] shift:  -4.733333333333352
IA1pm_Feb2_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  -9.8
IA1pm_Feb3_30Hz
Lever to [Ca] shift:  2.033333333333333
DLC to [Ca] shift:  4.1999999999999496
IA1pm_Feb4_30Hz
Lever to [Ca] shift:  2.4
DLC to [Ca] shift:  -10.0
IA1pm_Feb5_30Hz
Lever to [Ca] shift:  2.2666666666666666
DLC to [Ca] shift:  -9.533333333333335
IA1pm_Feb9_30Hz
Lever to [Ca] shift:  2.3333333333333335
DLC to [Ca] shift:  2.7999999999999545
IA1pm_Feb10_30Hz
Lever to [Ca] shift:  2.7333333333333334
DLC to [Ca] shift:  -4.466666666666686
IA1pm_Feb11_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.733333333333281
IA1pm_Feb12_30Hz
Lever to [Ca] shift:  2.466666666666667
DLC to [Ca] shift:  -0.46666666666670054
IA1pm_Feb15_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.266666666666616
IA1pm_Feb16_30Hz
Lever to [Ca] shift:  2.5
DLC to [Ca] shift:  4.599999999999948
IA1pm_Feb17_30Hz

IJ2pm_Feb4_30Hz
Lever to [Ca] shift:  2.3
DLC to [Ca] shift:  3.9999999999999503
IJ2pm_Feb5_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.399999999999949
IJ2pm_Feb9_30Hz
Lever to [Ca] shift:  2.2333333333333334
DLC to [Ca] shift:  4.066666666666617
IJ2pm_Feb10_30Hz
Lever to [Ca] shift:  2.1666666666666665
DLC to [Ca] shift:  3.666666666666618
IJ2pm_Feb11_30Hz
Lever to [Ca] shift:  2.6666666666666665
DLC to [Ca] shift:  4.599999999999948
IJ2pm_Feb12_30Hz
Lever to [Ca] shift:  2.433333333333333
DLC to [Ca] shift:  4.599999999999948
IJ2pm_Feb15_30Hz
Lever to [Ca] shift:  2.533333333333333
DLC to [Ca] shift:  4.266666666666616
IJ2pm_Feb16_30Hz
Lever to [Ca] shift:  3.2
DLC to [Ca] shift:  5.333333333333279
IJ2pm_Feb17_30Hz
IJ2pm_Feb18_30Hz
IJ2pm_Feb22_30Hz
IJ2pm_Feb23_30Hz
IJ2pm_Feb24_30Hz
IJ2pm_Feb25_30Hz
IJ2pm_Feb26_30Hz
IJ2pm_Feb29_30Hz
IJ2pm_Mar1_30Hz
IJ2pm_Mar2_30Hz
IJ2pm_Mar3_30Hz
IJ2am_Mar7_30Hz
IJ2pm_Mar8_30Hz
IJ2am_Mar9_30Hz
IJ2am_Mar10_30Hz
IJ2am_Mar11_30Hz


0 0 1 [ 1.33333333e-01 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -2.66666667e-01 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -3.55271368e-14 -3.55271368e-14  1.33333333e-01
 -3.55271368e-14 -6.66666667e-02]

0 0 2 [-2.66666667e-01 -3.55271368e-14 -6.66666667e-01 -2.66666667e-01
 -4.00000000e-01  2.66666667e-01 -2.00000000e-01  7.33333333e-01
 -2.66666667e-01 -2.66666667e-01 -2.66666667e-01 -1.33333333e-01
 -3.55271368e-14  8.66666667e-01 -1.33333333e-01 -3.55271368e-14
  2.66666667e-01  2.66666667e-01 -3.55271368e-14 -3.55271368e-14
 -1.33333333e-01  2.66666667e-01 -3.55271368e-14 -3.55271368e-14
 -3.55271368e-14 -1.33333333e-01 -3.55271368e-14 -3.55271368e-14
  1.33333333e-01]

0 0 3 [-3.55271368e-14 -

1 1 2 [-1.01923237 -3.52651398  0.12005946 -0.10683464 -0.75279353 -0.0253124
  0.36311001  0.30762951 -0.40021119 -0.75904954  0.09374976  0.11530218]

1 1 3 [-0.27441392 -0.90473067 -2.90962118  0.01503799 -0.37417175 -0.39644732
 -0.42632078 -0.11909424  0.40353845 -2.15570916  0.38519265 -1.76226368]

1 1 4 [-0.20998118 -0.37057944 -0.44379132 -0.18056835 -0.07495841 -0.24926844
 -0.12897531  0.05088769 -0.17129844 -0.49799539 -0.22302022]

1 1 5 [-0.25798218 -0.85351558 -2.80529177 -0.36813212  0.03942286 -0.25346105
  0.04150899  0.27282508  0.05942363 -2.3684086   0.03849474 -2.25401303]

1 1 6 [-0.64654654  2.54120454 -0.38663346  0.12050109  0.32944867 -0.33804299
  0.11361698  0.47106383 -0.47942764  0.68607449 -1.04610972]

1 2 3 [-6.59681616e-01 -1.78566630e+00  5.51851944e-02 -2.53453698e-02
  1.44348877e-01 -4.33165678e-04  7.42453222e-02 -1.11350440e-01
 -2.19375213e-01 -4.50789432e-01  1.72020484e-01  1.27293452e-01]

1 2 4 [-0.96681269 -0.1693804   0.27351162  0.579926

 -0.13333333 -2.53333333  0.26666667 -2.13333333]

4 2 6 [-2.13333333e+00  2.00000000e-01 -3.55271368e-14  2.00000000e-01
  2.66666667e-01  6.66666667e-02 -4.66666667e-01 -2.66666667e-01
 -2.40000000e+00]

4 3 4 [ 1.33333333e-01 -2.66666667e-01 -3.55271368e-14  1.26666667e+00
  6.66666667e-02  2.00000000e-01 -3.55271368e-14 -3.55271368e-14
  6.66666667e-02 -2.00000000e-01 -2.66666667e-01]

4 3 5 [0.33333333 0.13333333 0.13333333 0.13333333 0.13333333 1.26666667
 0.66666667 0.46666667 0.26666667 1.        ]

4 3 6 [-3.55271368e-14 -2.73333333e+00 -3.55271368e-14 -1.33333333e-01
  4.66666667e-01 -1.33333333e-01 -1.33333333e-01 -2.66666667e-01
  2.00000000e-01 -1.53333333e+00]

4 4 5 [-3.06666667e+00  2.66666667e-01 -3.55271368e-14 -2.66666667e-01
 -3.55271368e-14  6.00000000e-01  5.33333333e-01  3.33333333e-01
  3.33333333e-01 -3.13333333e+00]

4 4 6 [-2.86666667e+00 -4.00000000e-01  4.66666667e-01 -1.33333333e-01
 -3.55271368e-14 -1.33333333e-01 -3.55271368e-14  1.33333333e-01
 -3.55271

In [210]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

def diagonalize(data):
    for k in range(data.shape[0]):
        for p in range(0,k,1):
            data[k,p]=np.nan
    return data
            
fig = plt.figure(figsize=(12,12))
ax=plt.subplot(111)
data = correlated[:-1,:-1].astype('float32')
data = diagonalize(data)


data2 = uncorrelated[:-1,:-1].astype('float32')
data2 = diagonalize(data2)

im = ax.imshow(data/(data+data2), extent = [0,6,0,6])

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)

plt.colorbar(im, cax=cax)

ax.set_xticks(np.arange(6)+0.5)
ax.set_xticklabels(labels[1:7],rotation=45,fontsize=20)

ax.set_yticks(np.arange(6)+0.5)
ax.set_yticklabels(labels[0:6],rotation=0,fontsize=20)



# # 
# ax=plt.subplot(122)
# data = uncorrelated[:-1,:-1].astype('float32')
# data = diagonalize(data)
# im = ax.imshow(data, extent = [0,6,0,6])

# # create an axes on the right side of ax. The width of cax will be 5%
# # of ax and the padding between cax and ax will be fixed at 0.05 inch.
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)

# ax.set_xticks(np.arange(6)+0.5)
# ax.set_xticklabels(labels[1:7],rotation=45,fontsize=12)


#ax.xaxis.set_xticks(xticks)

plt.colorbar(im, cax=cax)

if True:
    plt.savefig('/home/cat/correlated_bodyparts_matrices.png',dpi=100)
    plt.close()
else:
    plt.show() 
    

In [202]:
fig = plt.figure()
vmin = np.min(rejected)
vmax = np.max(rejected)
print (vmin,vmax)
plt.imshow(rejected)
plt.show()

0 23


In [ ]:
fname = '/home/cat/concatenated_trials.npy'
svm  = PredictSVMTime()
svm.fname = fname

In [29]:
svm.process_data2()

X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (196, 900, 20)
X_l:  (22, 900, 20)
X_l:  (197, 900, 20)
X_l:  (21, 900, 20)
X_l:  (197, 900, 20)
X_l:  (21, 900, 20)


In [30]:
data = np.load('/home/cat/concatenated_res.npz')

cf = data['conf_matrix_ten']
print (cf.shape)

(10, 30, 30)


In [22]:
res = cf.mean(0)

plt.imshow(res)

plt.show()

In [ ]:
######################################################
##### PREDICT SVM DECISION CHOICE SINGLE SESSION #####
######################################################

# select animal names
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2'] # "AR4" and other datasets could work
#names = ['IA1']

# 
svm = PredictSVMChoice()
svm.root_dir = data_dir
svm.random_flag = False  # shuffle data to show baseline
svm.verbose = True

# window parameters
svm.window = 15              # prediction window, backwards in time in seconds
svm.lockout_window = 10      # time for locking out other pulls; OPTIONAL
svm.sliding_window = 30      # number of frames in sliding window for SVM steps; usually 1 sec = 30 frames
svm.lockout = False

# movement related params
svm.no_movement = 3          # number of seconds of quiescence before behavior initiation; parameter comes from
                             #     the "generate_movements_quiescent_periods" notebook in behavior_analysis
svm.random_lockout = 3       # number of seconds of buffer for selecting random trials for control

# pca params
svm.pca_flag = True
svm.pca_var = 0.95      # this is total amount of var explained to be used - NOT USED FOR NOW
svm.all_comps = 30      # this is the total # components saved
svm.nComp = 30          # this is the total # of comps to be used for decoding
            
# svm parameters
svm.min_trials = 10     # minimum number of trials to run SVM
svm.xvalidation = 10    # KFold xvalidation step
svm.data_split = 0.8    # split 80/20 data for training  NOT USED WITH XVALIDATION WHICH SPLITS 90/10
svm.method = 'sigmoid'  # method used for computing SVM

# run-time parameters
svm.parallel = True
svm.n_cores = svm.xvalidation
svm.overwrite = False

## session info
# session_ids = ['Mar1_', 'Mar2_', 'Mar3_', 'Feb29', 'Mar7_']
# svm.session_id = 'all'
svm.session_id = 'all'

# 
# codes = ['left_paw','right_paw','jaw']
labels = ['left_paw', 
          'right_paw',
          'nose',
          'jaw',
          'right_ear',
          'tongue',
          'lever',
          'all',
          'code_04',
          'code_04_lockout']

svm.labels = labels
svm.code = '_whole_stack'

# 
for name in names:
    svm.animal_id = name

    #
    svm.predict_whole_stack()   # this requires PCA to be run on the entire imaging session first: 400000, 128, 128
                                        # need also movement initiations 
                                        # need also shift between initiations and ca imaging
                                # This bypasses the need to compute [Ca] stms for each behavior/trial
